In [4]:
from tensorflow.python.summary.summary_iterator import summary_iterator

In [19]:
log_file1 = r'tb_logs\a\CCP\normal(0,5)\events.out.tfevents.1679904128.JesperPC.8972.1'
log_file2 = r'tb_logs\a\CCP\normal(0,5)\events.out.tfevents.1679904049.JesperPC.8972.0'
for e in summary_iterator(log_file2):
    #print(e.summary.value)
    for v in e.summary.value:
        print(v)
    break

In [27]:
import tensorboard as tb

In [29]:
experiment = tb.data.experimental.ExperimentFromDev(log_file2)

In [32]:
experiment.get_scalars()

_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Internal error. Please report this and specify reference code f1e44aad-e9e7-4b21-996c-952ed71a22cf."
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2023-05-08T15:25:39.744969422+00:00", grpc_status:13, grpc_message:"Internal error. Please report this and specify reference code f1e44aad-e9e7-4b21-996c-952ed71a22cf."}"
>

In [11]:
summary.summary.value

[]